In [34]:
#使うライブラリ
import numpy as np
from dataset.mnist import load_mnist

# データセットの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

# 4. ニューラルネットワークの学習

Let's study neural network! ではなくニューラルネットワークにデータを与えて学習させるほう。

## 4.1関連：特徴量という考え方

[特徴量とは、一言で表すと、分析対象データの中の、予測の手掛かりとなる変数のことです。](https://www.sbbit.jp/article/cont1/76066)  
また、データの何を変数として（特徴量として）使って機械学習を行おうかと、学習するためにデータを加工していく作業を特徴量エンジニアリング（feature engineering）ともいう。例えばデータセットの中に変数A,変数B,変数Cがあるとして、全てを機械学習アルゴリズムにかけるか、あるいはAとBだけというように一部だけ使うか、あるいは新たな変数を作るか（A＋BーC＝D的な）といった選択を行い、より精度の良いモデルを作ること。  
  
一般的な機械学習はデータ→特徴量を決める（特徴量エンジニアリング）→形の変わったデータ→機械学習の流れで行われる（粗い理解）。その一方でディープラーニングではその特徴量エンジニアリングがいらない。データをそのまま入力すると勝手にノードの重み（特徴量）を調整する。すごい。

## 4.2 損失関数

最適な重みパラメーターの探索のために用いられる関数。主に2乗和誤差や交差エントロピー誤差が用いられる。損失関数を使ってモデルの予測が教師データとどれだけズレているかを算出し、そのズレを小さくしていくことがニューラルネットワークの学習になる。

### 2乗和誤差

以下の計算でエラー(E)を求める。1/kをかけることもあるが、1/2とテキストでは記載されている。誤植ではなく[微分をしやすくするためらしい。](https://canplay-music.com/2019/06/02/loss-function/)  
$$
E = \frac{1}{2}\sum_{k}(y_k - t_k)^2
$$

In [8]:
def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)

In [16]:
# 例

##　yはニューラルネットからのソフトマックス関数を使っての出力のサンプル。1桁の数字の識別タスクと家庭。

## 正解ラベルのワンホットベクトルでの表現。「２」が正解のベクトル。
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

## ２を予測（せいかい）するベクトル。
y1 = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
## 7を予測（まちがい）するベクトル
y2 = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]

# 正解を予測できている場合
print("正解の予測の二乗和誤差の値")
print(mean_squared_error(np.array(y1), np.array(t)))
print()
print("間違いの予測の二乗和誤差の値")
print(mean_squared_error(np.array(y2), np.array(t)))



正解の予測の二乗和誤差の値
0.09750000000000003

間違いの予測の二乗和誤差の値
0.5975


上記のように、正解を出力できる方が間違いを出力する場合よりも二乗和誤差の数字が小さくなる。教師データ(t)により適合していることを示している。

### 交差エントロピー誤差

以下の計算で誤差（E）を求める。logの底は自然対数。
$$
E = -\sum_kt_k\log y_k
$$
ここで、tは正解ラベルのワンホット表現（正解のインデックス以外は0）なので、実際は正解のインデックスに対応する出力の自然対数を計算する関数となっている。$\log 1$は0なのでドンピシャに予測できると誤差がゼロになる。予測の確率が1より低くなると-で数値が大きくなる。たとえば$\log 0.5$はおよそ-0.69で$\log 0.1$はおよそ-2.3。マイナスを取るために総和のあとで-1がかけられている。

In [27]:
# 実装
def cross_entropy_error(y, t):
    delta = 1e-7 # log 0を割けるために微小な数字を計算時に足す
    return -np.sum(t * np.log(y + delta))

In [30]:
# 例

##　yはニューラルネットからのソフトマックス関数を使っての出力のサンプル。1桁の数字の識別タスクと家庭。

## 正解ラベルのワンホットベクトルでの表現。「２」が正解のベクトル。
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

## ２を予測（せいかい）するベクトル。
y1 = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
## 7を予測（まちがい）するベクトル
y2 = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]

# 正解を予測できている場合
print("正解の予測の交差エントロピー誤差の値")
print(cross_entropy_error(np.array(y1), np.array(t)))
print("間違いの予測の交差エントロピー誤差の値")
print(cross_entropy_error(np.array(y2), np.array(t)))


正解の予測の交差エントロピー誤差の値
0.510825457099338
間違いの予測の交差エントロピー誤差の値
2.302584092994546


二乗和誤差のときと同様に、間違いの予測の値のほうが大きくなっているのがわかる。

### 4.2.3ミニバッチ学習

大量にある訓練データをある程度のまとまり（ミニバッチ）にしてそれをひとまとまりとして、ミニバッチごとに損失関数を計算する。
これまで見てきた実装だとデータを一つ見て損失関数を計算していたが、ミニバッチだと大きな数のデータ（例えば100個とか）の損失関数を一度に計算するので、一つ一つ計算するよりも早い。  
なお、一つずつ訓練データを使って損失関数を計算する手法をオンライン学習（Udemyではない）という。逆にすべての訓練データを使っていっぺんに損失関数を計算する手法をバッチ学習（ミニではない）という。ミニバッチ学習はバッチ学習とオンライン学習の中間のイメージ。

In [51]:
#　ミニバッチ学習（たとえば100個纏めて計算）出来るように交差エントロピー誤差の実装を変える

def cross_entropy_error(y, t):
    #データセットが一つだけの時に1次元から2次元に変形する必要がある。
    if y.dim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

## 微分